In [1]:
# Author: Kai Zhang
# Stevens Institute of Technology

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time

executable_path = 'F:\geckodriver'

driver = webdriver.Firefox(executable_path=executable_path)

driver.get('https://steamcommunity.com/app/435150/reviews/?p=1&browsefilter=toprated')

src_updated = driver.page_source
src = ""

for i in range(0,190):
    if src != src_updated:
        # save page source (i.e. html document) before page-down
        src = src_updated
        # execute javascript to scroll to the bottom of the window
        # you can also use page-down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # sleep to allow content loaded
        time.sleep(1)
        # save page source after page-down
        src_updated = driver.page_source


# What can we get about the reviews from Steam?

# The review content, post date, how many people think this review helpful or funny

In [2]:
review_content=[]
usefuls=[]

for i in range(1,191):
    # get all Q&A list using XPATH locator
    lists=driver.find_elements_by_xpath("//div[@id='page%d']/div"%i)
#     print("//div[@id='page%d']/div"%i)
#     print("page%d pairs: "%i,len(lists))
    
    for idx,item in enumerate(lists):    
    # each Q&A pair has an unique ID
        div_id=item.get_attribute("id")
        content_css="div#"+div_id+" "+"div.apphub_UserReviewCardContent div.apphub_CardTextContent"
        useful_css="div#"+div_id+" "+"div.apphub_UserReviewCardContent div.found_helpful"
        
        review=driver.find_element_by_css_selector(content_css)
        useful=driver.find_element_by_css_selector(useful_css)
        review_content.append(review.text)
        usefuls.append(useful.text)

print("Total reviews scraped: ", len(review_content), len(usefuls))
# print(review_content[0])
# print(usefuls[0])

Total reviews scraped:  1198 1198


In [8]:
import pandas as pd
import nltk
import re

reviews =[]
dates = []

# print(type(data[0]),str(data[0]))

for line in review_content:
    sentence = line.split("\n")
    date = re.sub(r"Posted:"," ",sentence[0]).strip()
    review = re.sub(r"Posted:.*"," ",line)
    review = re.sub(r"\s+"," ",review).strip() 
    reviews.append(review)
    dates.append(date)
    
# print("date:",dates[0])
# print("content:",reviews[0])



df = pd.DataFrame()
df["review"] = reviews
df["post_date"] = dates
df["useful"] = usefuls
df.to_csv('game_review.csv',index=True)